In [1]:
"""
Use 2.x tensorflow version
"""
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


TensorFlow 2.x selected.
Found GPU at: /device:GPU:0


In [0]:
import numpy as np
import os,sys
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image


from skimage.transform import rotate, resize
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import backend as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [28]:
import keras 
keras.__version__

Using TensorFlow backend.


'2.2.5'

In [0]:
"""
This cells contains all the function needed

"""

def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

# Concatenate an image and its groundtruth
def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = np.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
        gt_img8 = img_float_to_uint8(gt_img)
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = np.concatenate((img8, gt_img_3c), axis=1)
    return cimg

# Convert array of labels to an image

def label_to_img(imgwidth, imgheight, w, h, labels):
    im = np.zeros([imgwidth, imgheight])
    idx = 0
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            im[j:j+w, i:i+h] = labels[idx]
            idx = idx + 1
    return im

def value_to_class(v, foreground_threshold=0.25):

    df = np.sum(v)
    if df > foreground_threshold:
        return 1
    else:
        return 0


def rotate_images(X, Y , degrees):
    """
    increase the number of data
    by adding rotations of the base data
    """

    X = np.array(X)
    Y = np.array(Y)
    rotimg = np.zeros(X.shape)
    rotgtimg = np.zeros(Y.shape)

    Xtemp = X
    Ytemp = Y

    #rotate all images by degree and add them to the data vector
    for degree in degrees:
        for i in range(len(Xtemp)):
            rotimg[i] = rotate(Xtemp[i], degree, resize=False, mode='reflect')
            rotgtimg[i] = rotate(Ytemp[i], degree, resize=False, mode='reflect')
        X = np.concatenate([X,rotimg])
        Y = np.concatenate([Y,rotgtimg])

    return X,Y

def resize_image(X, Y, size = 400):

    X = np.asarray(X)
    Y = np.asarray(Y)

    Xresize = np.asarray([resize(X[i], (size,size), mode = 'reflect') for i in range(X.shape[0])])
    Yresize = np.asarray([resize(Y[i], (size,size), mode = 'reflect') for i in range(X.shape[0])])

    return Xresize, Yresize


def create_submission(y_pred, filename = 'filename', patch_size = 16, img_size = 608):
    """
    Create a submission (csv format) for AIcrowd
    from given prediction

    """
    n = img_size // patch_size
    y_pred = np.reshape(y_pred, (-1, n, n))

    with open(filename, 'w') as f:
        f.write('id,prediction\n')
        for i in range(y_pred.shape[0]):
            img = y_pred[i]
            for j in range(img.shape[0]):
                for k in range(img.shape[1]):
                    name = '{:03d}_{}_{},{}'.format(i + 1, j * patch_size, k * patch_size, int(img[j,k]))
                    f.write(name + '\n')


def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches


def load_trainset(path = 'Data/test_set_images'):

    # Loaded a set of images
    root_dir = path

    image_dir = root_dir + "images/"
    files = os.listdir(image_dir)
    n = len(files) # Use all images
    print("Loading " + str(n) + " testing images")
    imgs = [load_image(image_dir + files[i]) for i in range(100)]

    gt_dir = root_dir + "groundtruth/"
    print("Loading " + str(n) + " groundtruth")
    gt_imgs = [load_image(gt_dir + files[i]) for i in range(100)]

    return imgs, gt_imgs


def load_testset(path = 'Data/test_set_images'):


  root_testdir = path
  test_names = os.listdir(root_testdir)

  num_test = len(test_names)
  order = [int(test_names[i].split("_")[1]) for i in range(num_test)]
  index = np.argsort(order)

  # Load image and reorder them
  imgs_test = [load_image(os.path.join(root_testdir, test_names[i], test_names[i]) + ".png")
               for i in range(num_test)]
  imgs_test = [imgs_test[i] for i in index]

  return imgs_test

def XYaugmentGenerator(X1, y, gen, seed = 1, batch_size = 1):
    genX1 = gen.flow(X1, y, batch_size=batch_size, seed=seed)
    genX2 = gen.flow(y, X1, batch_size=batch_size, seed=seed)
    while True:
        X1i = genX1.next()
        X2i = genX2.next()

        yield X1i[0], X2i[0]

def test_image_unet_submission(imgs_test, model, size = 400, foreground_threshold = 0.25, filename = 'submission.csv'):


  img1 = []
  img2 = []
  img3 = []
  img4 = []

  shift = 608 - size

  for img in imgs_test:
    img = img[0:size, 0:size, :]
    img1.append(img)

  for img in imgs_test:
    img = img[shift:608, 0:size, :]
    img2.append(img)

  for img in imgs_test:
    img = img[0:size, shift:608, :]
    img3.append(img)

  for img in imgs_test:
    img = img[shift:608, shift:608, :]
    img4.append(img)

  img_pred1 = model.predict(np.asarray(img1), batch_size = 1, verbose = 1)
  img_pred2 = model.predict(np.asarray(img2), batch_size = 1, verbose = 1)
  img_pred3 = model.predict(np.asarray(img3), batch_size = 1, verbose = 1)
  img_pred4 = model.predict(np.asarray(img4), batch_size = 1, verbose = 1)


  img_pred1[img_pred1 <= 0.5] = 0
  img_pred1[img_pred1 > 0.5] = 1

  img_pred2[img_pred2 <= 0.5] = 0
  img_pred2[img_pred2 > 0.5] = 1

  img_pred3[img_pred3 <= 0.5] = 0
  img_pred3[img_pred3 > 0.5] = 1

  img_pred4[img_pred4 <= 0.5] = 0
  img_pred4[img_pred4 > 0.5] = 1

  img1 = np.asarray(img1)
  img2 = np.asarray(img2)
  img3 = np.asarray(img3)
  img4 = np.asarray(img4)

  list_m1 = []
  list_m2 = []
  list_merge = []

  for i in range(50):
    m1 = np.concatenate((img_pred1[i, 0:shift, :, :], img_pred2[i, :, :, :]), axis = 0)
    list_m1.append(m1)
    m2 = np.concatenate((img_pred3[i, 0:shift, :, :], img_pred4[i, :, :, :]), axis = 0)
    list_m2.append(m2)

  list_m1 = np.asarray(list_m1)
  list_m2 = np.asarray(list_m2)

  for i in range(50):
    merge = np.concatenate((list_m1[i,:,:,:], list_m2[i, :, (size - shift):size, :]), axis=1)
    list_merge.append(merge)

  y_pred = np.asarray(list_merge)

  pred_patch = [img_crop(y_pred[i], 16, 16) for i in range(y_pred.shape[0])]
  pred_patch = np.asarray([pred_patch[i][j] for i in range(len(pred_patch)) for j in range(len(pred_patch[i]))])
  pred_patch = np.asarray([value_to_class(np.mean(pred_patch[i]), foreground_threshold=foreground_threshold) for i in range(pred_patch.shape[0])])

  create_submission(pred_patch, filename)


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
imgs, gt_imgs = load_trainset(path = 'drive/My Drive/training/')

Loading 100 testing images
Loading 100 groundtruth


In [5]:
root_testdir = "drive/My Drive/test_set_images"
test_names = os.listdir(root_testdir)

num_test = len(test_names)
order = [int(test_names[i].split("_")[1]) for i in range(num_test)]
index = np.argsort(order)

imgs_test = [load_image(os.path.join(root_testdir, test_names[i], test_names[i]) + ".png") 
             for i in range(num_test)]
imgs_test = [imgs_test[i] for i in index]
print('Test set loaded')

Test set loaded


In [0]:
imgs, gt_imgs = rotate_images(imgs, gt_imgs, [ 15, 30, 45, 60, 75])

n = len(imgs)

In [0]:
SEED = 2019
np.random.seed(SEED)
tf.compat.v1.set_random_seed(SEED)

In [0]:
img_size = 400

Xr, Yr = resize_image(imgs, gt_imgs, img_size)
Yr = np.reshape(Yr, (n, img_size, img_size, 1))

In [0]:
x_train, x_val, y_train, y_val = train_test_split(Xr, Yr, test_size=0.1,  random_state=SEED)

In [0]:
def unet(input_size = (400,400,3)):
    inputs = Input(input_size)
    conv1 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.25)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(2048, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(2048, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(1024, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs, conv10)

    return model


In [0]:
input_size = (400, 400, 3)
model = unet((400, 400, 3))

#model.summary()

In [0]:
# Create image generator
data_gen_args = dict(
        width_shift_range=0.05,
        height_shift_range=0.05,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,)


image_datagen = ImageDataGenerator(**data_gen_args, fill_mode ='reflect')

train_gen = XYaugmentGenerator(x_train, y_train, image_datagen)

In [0]:
model_filename = 'model_name.h5'

callback_checkpoint = ModelCheckpoint(model_filename, verbose=1, monitor='val_loss', save_best_only=True)
lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, verbose=1, mode='min', min_lr= 1e-8)
es = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min')


In [14]:
model.compile(loss='binary_crossentropy', 
              optimizer= Adam(lr = 1e-4),
              metrics=['binary_accuracy'])

history = model.fit(
    train_gen,
    steps_per_epoch=540,
    epochs=120,   
    validation_data=(x_val, y_val),
    callbacks=[callback_checkpoint, es, lr]
)

  ...
    to  
  ['...']
Train for 540 steps, validate on 60 samples
Epoch 1/120
 30/540 [>.............................] - ETA: 4:55 - loss: 0.5374 - binary_accuracy: 0.7337WARNING:tensorflow:Can save best model only with val_loss available, skipping.


KeyboardInterrupt: ignored

In [0]:
model.save_weights(filepath='drive/My Drive/model_name.h5')

In [0]:
"""
Loaded our best model, the model can be downloaded from the link in the readme and has to be
upload in your drive
"""
model.load_weights('drive/My Drive/lastrun.h5')


In [0]:
"""
Run this cell if you want to plot loss/val_loss from the previous training
"""

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [15]:
test_image_unet_submission(imgs_test, model, filename = 'drive/My Drive/submission.csv', foreground_threshold = 0.25)

30/50 [=================>............] - ETA: 2s

KeyboardInterrupt: ignored